In [ ]:
!pip install pandas
!pip install gdown

In [1]:
import pandas as pd
import numpy as np


# Descargar datos limpios
Inventario de vivienda
Contiene la información que da cuenta de las viviendas y el entorno urbano en el que se están desarrollando.

Año:
2017 al 2023

- [Link](https://sniiv.sedatu.gob.mx/Reporte/Datos_abiertos)
- [Diccionario](https://sniiv.sedatu.gob.mx/Reporte/Datos_abiertos#:~:text=Diccionario%20de%20datos)


In [ ]:
!git clone https://github.com/javierIA/infonavit.git

In [ ]:
#@title UnirVarios DTF CON EL mismo formato
import pandas as pd
import os

# Create an empty list to store the dataframes
dfs = []

# Iterate over each CSV file in the "Datos" directory
for file_name in os.listdir("infonavit"):
    if file_name.endswith(".csv"):
        file_path = os.path.join("infonavit", file_name)
        df = pd.read_csv(file_path, encoding="latin1")
        dfs.append(df)

# Concatenate all dataframes into a single dataframe
merged_df = pd.concat(dfs, ignore_index=True)

# Convert merged dataframe to Parquet format
output_file = "merged_data.parquet"
merged_df.to_parquet(output_file, engine="pyarrow")

print(f"Data converted to Parquet format and saved as {output_file}")


In [ ]:
db=pd.read_parquet("/content/merged_data.parquet")

db.sample(10)


# Llenar un columna con datos vacios de otra

In [ ]:
db.describe().T

In [ ]:
valores_nulos = db.isnull().sum()
porcentajes_nulos = valores_nulos / db.shape[0] * 100
porcentajes_nulos_ordenados = porcentajes_nulos.sort_values(ascending=False)
porcentajes_nulos_ordenados

In [ ]:
db['municipio'] = db['municipio'].fillna(db['minicipio']) # llena las columnas faltantes de municipio
db['año'] = db['año'].fillna(db['aÃ±o']) # llena las columnas de año
db.drop(['aÃ±o','minicipio'], axis=1, inplace=True)
db

# Re ordenar las columnas de acuerdo orden original


In [ ]:
column_order = ['año', 'municipio', 'mes', 'cve_ent', 'entidad', 'cve_mun', 'organismo', 'modalidad', 'destino', 'tipo', 'sexo', 'edad_rango', 'ingresos_rango', 'vivienda_valor', 'acciones', 'monto']

db = db.reindex(columns=column_order)

In [ ]:
db.info()

# Diccionario

Para reemplazar los códigos de rango de edad con sus descripciones correspondientes en la columna 'edad_rango', puedes utilizar el método replace de pandas. Aquí tienes un ejemplo de cómo hacerlo

In [ ]:
# Diccionario para la edad
age_dict = {
    1: '29 o menos',
    2: '30 a 59',
    3: '60 o más',
    4: 'No especificado'
}

db['edad_rango'] = db['edad_rango'].replace(age_dict)
db.edad_rango

In [ ]:
# Supongamos que 'merged_df' es el dataframe combinado
income_dict = {
    1: '2.6 o menos',
    2: '2.61 a 4.00',
    3: '4.01 a 6.00',
    4: '6.01 a 9.00',
    5: '9.01 a 12.00',
    6: 'Más de 12',
    7: 'No especificado'
}

# Reemplazar los códigos de rango de ingresos con las descripciones correspondientes
db['ingresos_rango'] = db['ingresos_rango'].replace(income_dict)

# Imprimir el dataframe actualizado
print(db)


Para reemplazar los códigos de valor de vivienda con sus descripciones correspondientes en la columna 'vivienda_valor', puedes utilizar el método replace de pandas. Aquí tienes un ejemplo de cómo hacerlo:



In [ ]:
housing_dict = {
    1: 'Económica',
    2: 'Popular',
    3: 'Tradicional',
    4: 'Media',
    5: 'Residencial',
    6: 'Residencial plus',
    7: 'No especificado'
}

db['vivienda_valor'] = db['vivienda_valor'].replace(housing_dict)

print(db)


In [ ]:
organism_dict = {
    1: "INFONAVIT",
    2: "CNBV",
    3: "FOVISSSTE",
    4: "SHF",
    5: "CONAVI",
    6: "INVI",
    7: "Banjercito",
    8: "Hábitat México",
    9: "CFE",
    10: "ISSFAM",
    11: "PEMEX",
    12: "FONHAPO",
    13: "PDZP SEDESOL",
    14: "SOFOLES AMFE",
    15: "ISSSTELEON",
    16: "INDIVI",
    17: "COVEG",
    18: "COESVI",
    19: "IMEVIS",
    20: "IVEM",
    21: "ITAVU",
    22: "IVNL",
    23: "INVIVIENDA",
    24: "IVEY",
    25: "INFOVIR",
    26: "Instituto Nacional del Suelo Sustentable"
}

db['organismo'].replace(organism_dict, inplace=True)

print(db)


Graficar con m

In [ ]:
# creamos el archivo data.csv
db.to_csv("data.csv")
db.head(10)

In [ ]:
db.shape



---
# Haz un histograma de la columna 'año'



In [ ]:
import matplotlib.pyplot as plt

# Haz un histograma de la columna 'año'
db['año'].hist(bins=30, edgecolor='black')

# Añade títulos y etiquetas
plt.title('Histograma de Años')
plt.xlabel('Año')
plt.ylabel('Frecuencia')


In [ ]:
# Visualizar tabla previo a la grafica
print(db['organismo'].value_counts())

# Elabora la grafica
db['organismo'].value_counts().plot(kind='bar')
plt.title('Número de Acciones por Organismo')
plt.xlabel('Organismo')
plt.ylabel('Número de Acciones')
plt.show()


In [ ]:
data_grouped = db.groupby('año')['monto'].mean()

# Dibuja el gráfico de líneas
plt.figure(figsize=(10,6))
plt.plot(data_grouped.index, data_grouped.values, marker='o')

# Añade títulos y etiquetas
plt.title('Variacion en el Monto a lo largo de los Años')
plt.xlabel('Año')
plt.ylabel('Monto Promedio')

# Muestra el gráfico
plt.show()

In [ ]:
db.shape

# Se duplico el gasto de costo de vivienda